<a href="https://colab.research.google.com/github/khojun611/6team/blob/main/DataAugmentation_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [72]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# 패키지 임포트

In [73]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from pathlib import Path
from tqdm import tqdm_notebook

import cv2
from numpy import expand_dims
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [71]:
train_df = pd.read_csv('/content/drive/MyDrive/DACON/train_df.csv')
train_df.head()

,index,file_name,class,state,label
0,0,10000.png,transistor,good,transistor-good
1,1,10001.png,capsule,good,capsule-good
2,2,10002.png,transistor,good,transistor-good
3,3,10003.png,wood,good,wood-good
4,4,10004.png,bottle,good,bottle-good


# 품목별, 상태별로 나누어 data_train_state 폴더에 저장

In [76]:
filepath = "/content/drive/MyDrive/DACON/train"

classes = os.listdir(filepath)

# 사물의 상태 저장할 상위 폴더
dir_path = "/content/drive/MyDrive/DACON/data_train_state"

# dir_path 폴더 없으면 생성
if os.path.isdir(dir_path) == False:
    os.mkdir(dir_path)

for i, cls in enumerate(classes):
    image = Image.open(filepath + '/' + cls)
    
    filter1 = (train_df['file_name'] == cls)
    df = train_df[filter1]
    class1 = df['class'].values[0]

    # 클래스별 폴더 경로
    folder_path = dir_path + '/' + class1
    
    # 폴더 없으면 생성
    if os.path.isdir(folder_path) == False:
        os.mkdir(folder_path)
    
    # 사물의 상태값
    state = df['state'].values[0]
    
    # 상태별 폴더 경로
    state_path = folder_path + '/' + state
    
    print(state_path)
    
    # 상태별 폴더 없으면 생성
    if os.path.isdir(state_path) == False:
        os.mkdir(state_path)
        
    # 이미지 저장
    image.save(state_path + '/' + cls, format = None)

/content/drive/MyDrive/DACON/data_train_state/bottle/good
/content/drive/MyDrive/DACON/data_train_state/zipper/good
/content/drive/MyDrive/DACON/data_train_state/pill/good
/content/drive/MyDrive/DACON/data_train_state/metal_nut/bent
/content/drive/MyDrive/DACON/data_train_state/capsule/good
/content/drive/MyDrive/DACON/data_train_state/zipper/good
/content/drive/MyDrive/DACON/data_train_state/zipper/good
/content/drive/MyDrive/DACON/data_train_state/bottle/good
/content/drive/MyDrive/DACON/data_train_state/hazelnut/good
/content/drive/MyDrive/DACON/data_train_state/hazelnut/good
/content/drive/MyDrive/DACON/data_train_state/tile/good
/content/drive/MyDrive/DACON/data_train_state/tile/good
/content/drive/MyDrive/DACON/data_train_state/wood/good
/content/drive/MyDrive/DACON/data_train_state/capsule/good
/content/drive/MyDrive/DACON/data_train_state/transistor/good
/content/drive/MyDrive/DACON/data_train_state/tile/good
/content/drive/MyDrive/DACON/data_train_state/hazelnut/good
/content/

In [77]:
base_path = "/content/drive/MyDrive/DACON/data_train_state/"
item_list = os.listdir(base_path)

print(item_list)
print(len(item_list))

['bottle', 'zipper', 'pill', 'metal_nut', 'capsule', 'hazelnut', 'tile', 'wood', 'transistor', 'cable', 'leather', 'carpet', 'screw', 'grid', 'toothbrush']
15


In [78]:
filepath = []

for X in item_list:
  filepath.append(base_path + X + "/")


print(filepath)
print(len(filepath))

['/content/drive/MyDrive/DACON/data_train_state/bottle/', '/content/drive/MyDrive/DACON/data_train_state/zipper/', '/content/drive/MyDrive/DACON/data_train_state/pill/', '/content/drive/MyDrive/DACON/data_train_state/metal_nut/', '/content/drive/MyDrive/DACON/data_train_state/capsule/', '/content/drive/MyDrive/DACON/data_train_state/hazelnut/', '/content/drive/MyDrive/DACON/data_train_state/tile/', '/content/drive/MyDrive/DACON/data_train_state/wood/', '/content/drive/MyDrive/DACON/data_train_state/transistor/', '/content/drive/MyDrive/DACON/data_train_state/cable/', '/content/drive/MyDrive/DACON/data_train_state/leather/', '/content/drive/MyDrive/DACON/data_train_state/carpet/', '/content/drive/MyDrive/DACON/data_train_state/screw/', '/content/drive/MyDrive/DACON/data_train_state/grid/', '/content/drive/MyDrive/DACON/data_train_state/toothbrush/']
15


# 증강시킬 이미지 경로 추출

In [79]:
folder = []
for i in range(len(item_list)):
  folder.append(os.listdir(filepath[i]))

all_path = []

for i in range(len(item_list)):
  for j in range(len(folder[i])):
    all_path.append(filepath[i] + folder[i][j])

df = pd.DataFrame(all_path)
index_list = df[df[0].str.endswith('good')].index
df = df.drop(index_list)

aug_path = df[0].values.tolist()

print(len(all_path))  #number of folders including 'good'
print(len(aug_path))  #number of folders except for 'good'

88
73


In [80]:
img_num = []

for i in range(len(aug_path)):
  img_num.append(os.listdir(aug_path[i]))

img_path = []

for i in range(len(aug_path)):
  for j in range(len(img_num[i])):
    img_path.append(aug_path[i] + '/' + img_num[i][j])

print(img_path[0])
print(len(img_path))  #number of images to be augmented

/content/drive/MyDrive/DACON/data_train_state/bottle/broken_small/13436.png
648


# 이미지 증강

In [81]:
def create_image(gen_data, filepath, imgName):
    samples = expand_dims(gen_data, 0)
    datagen = ImageDataGenerator(rotation_range = 10, horizontal_flip=True)

    it = datagen.flow(samples, batch_size = 1)
    #fig = plt.figure(figsize = (224, 224))

    for i in range(9): # 이미지 한 장당 9배
        #plt.subplot(3, 3, i + 1)

        batch = it.next()
        image = batch[0].astype('uint8')
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        cv2.imwrite(filepath + "/aug_" + str(imgName)[-5:] + "_" + str(i) + ".png", image)

In [82]:
for i in range(len(img_path)):
  img = img_path[i]
  path = Path(img_path[i]).parent
  imgName = os.path.splitext(img)[0]
            
  genimg = load_img(img) # 데이터 가져오기
  gen_data = img_to_array(genimg)
  create_image(gen_data, str(path), imgName)